In [1]:
import pandas as pd
train_df = pd.read_csv("../sales_train.csv.gz")
test_df  = pd.read_csv("../test.csv.gz")

In [2]:
%store -r __prepare_data

In [3]:
__prepare_data

In [8]:
# more lags for mean encoding
trg_means = all_data.groupby(["item_id","date_block_num"]).target.mean()
trg_means.name = "item_target_enc"
trg_lag_2 = trg_means.reset_index("date_block_num") # this turns it into a DataFrame
trg_lag_2["date_block_num"] = trg_lag_2["date_block_num"] + 2
trg_lag_2.set_index('date_block_num', append=True, inplace=True)
trg_lag_2.columns = ["item_target_enc_lag_2"]
all_data = all_data.merge(trg_lag_2.reset_index(), how="left", on=["item_id","date_block_num"])

trg_means = all_data.groupby(["shop_id","date_block_num"]).target.mean()
trg_means.name = "shop_target_enc"
trg_lag_2 = trg_means.reset_index("date_block_num")
trg_lag_2["date_block_num"] = trg_lag_2["date_block_num"] + 2
trg_lag_2.set_index('date_block_num', append=True, inplace=True)
trg_lag_2.columns = ["shop_target_enc_lag_2"]
all_data = all_data.merge(trg_lag_2.reset_index(), how="left", on=["shop_id","date_block_num"])

In [9]:
all_data["item_target_enc_lag_ma_1_2"] = (all_data.loc[:,"item_target_enc_lag_1"] + all_data.loc[:,"item_target_enc_lag_2"])/2.
all_data["shop_target_enc_lag_ma_1_2"] = (all_data.loc[:,"shop_target_enc_lag_1"] + all_data.loc[:,"shop_target_enc_lag_2"])/2.

In [10]:
all_data = all_data[all_data.date_block_num > 1] # no lags before time 1
all_data.fillna(0, inplace=True)
print(all_data.isnull().values.any())

False


In [17]:
from sklearn.model_selection import train_test_split

target = all_data.target.values
target[target>20] = 20

predictors = ["target_lag_1","item_target_enc_lag_1","shop_target_enc_lag_1",
                             "item_target_enc_lag_2","shop_target_enc_lag_2",
                             "item_target_enc_ave",  "shop_target_enc_ave"]

X_train, X_test, y_train, y_test = \
    train_test_split(all_data[predictors], target, test_size=0.2, random_state=123)

In [18]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

model2 = RandomForestRegressor(n_estimators=64, max_features=2, n_jobs=6) #len(predictors)/2
model2.fit(X_train, y_train)
ypred = model2.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, ypred))
print("RMSE: %f" % (rmse))

RMSE: 0.786600


In [19]:
from sklearn.ensemble import RandomForestRegressor
full_model = RandomForestRegressor(n_estimators=64, max_features=2, n_jobs=6) #len(predictors)/2
full_model.fit(all_data[predictors], target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=2, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=64, n_jobs=6,
                      oob_score=False, random_state=None, verbose=0,
                      warm_start=False)

In [20]:
%store -r __prepare_submission

In [21]:
__prepare_submission

In [25]:
aliases = \
{"target_lag_1":          "target",
 "item_target_enc_lag_1": "item_target_enc",
 "shop_target_enc_lag_1": "shop_target_enc",
 "item_target_enc_lag_2": "item_target_enc_lag_1",
 "shop_target_enc_lag_2": "shop_target_enc_lag_1",
 "item_target_enc_ave":   "item_target_enc_ave",
 "shop_target_enc_ave":   "shop_target_enc_ave"
 }
all_data_33.fillna(0, inplace=True)
prepare_submission(full_model, predictors, aliases, "skl_rf_test.csv")

In [23]:
X = all_data_33.loc[:,[aliases[colname] for colname in predictors]]

In [24]:
X.head()

,target,item_target_enc,shop_target_enc,item_target_enc_lag_1,shop_target_enc_lag_1,item_target_enc_ave,shop_target_enc_ave
10675678,0.0,0.022727,0.134306,0.069767,0.161652,1.355613,0.130904
10675679,1.0,0.409091,0.134306,0.209302,0.161652,0.931213,0.130904
10675680,0.0,0.500000,0.134306,0.441860,0.161652,1.319042,0.130904
10675681,0.0,0.363636,0.134306,0.372093,0.161652,0.527112,0.130904
10675682,0.0,0.022727,0.134306,NaN,0.161652,0.070943,0.130904
